In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd



[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\rathi\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
data=gutenberg.raw('shakespeare-hamlet.txt')
#save to a file
with open('hamlet.txt','w') as file:
    file.write(data)

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [4]:
with open('hamlet.txt','r') as file:
    text=file.read().lower()


#tokenize text

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [5]:
inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range (1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inputsequences.append(n_gram_sequence)


In [6]:
#padding
max_sequence_length=max([len(x) for x in inputsequences])
max_sequence_length

14

In [7]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_sequence_length,padding='pre'))
display(inputsequences)

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [8]:
x, y = inputsequences[:, :-1], inputsequences[:, -1]

In [9]:
import tensorflow as tf
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
display(y)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
import numpy as np
import pickle

# Prepare vocabulary size and input length
total_words = len(tokenizer.word_index) + 1
max_sequence_length = max([len(x) for x in inputsequences])

# Build the model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length - 1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Force build the model with dummy input so summary works
model(np.zeros((1, max_sequence_length - 1), dtype=np.int32))

# Print model summary
model.summary()

c:\Users\rathi\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 13, 100)           │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 13, 150)           │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (1, 13, 150)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 100)               │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 4818)              │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 13, 100)           │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 13, 150)           │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (1, 13, 150)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 100)               │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 4818)              │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
##train the moodel
history=model.fit (x_train, y_train, epochs=50,validation_data= (x_test,y_test) , verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 72s 89ms/step - accuracy: 0.0322 - loss: 7.1157 - val_accuracy: 0.0313 - val_loss: 6.7854
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 44s 69ms/step - accuracy: 0.0379 - loss: 6.4323 - val_accuracy: 0.0418 - val_loss: 6.8620
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 51s 79ms/step - accuracy: 0.0416 - loss: 6.2850 - val_accuracy: 0.0482 - val_loss: 6.9275
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 48s 74ms/step - accuracy: 0.0513 - loss: 6.1642 - val_accuracy: 0.0493 - val_loss: 6.9536
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 52s 80ms/step - accuracy: 0.0536 - loss: 6.0564 - val_accuracy: 0.0503 - val_loss: 6.9867
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.0627 - loss: 5.9147 - val_accuracy: 0.0563 - val_loss: 7.0497
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 50s 77ms/step - accuracy: 0.0745 - loss: 5.7687 - val_accuracy: 0.0602 - val_loss: 7.0815
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 53s 83ms/step - accuracy: 0.0802 - loss: 5.6539 - 

In [20]:
def predict_next_word(model, tokenizer, text, max_sequence_length):
    # Tokenize and pad input text
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = token_list[-(max_sequence_length - 1):]  
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

    # Predict next word
    predicted = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted)


    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word
    
    # If index not found, fallback
    return f"[Unknown index: {predicted_index}]"

In [21]:
input_text="To be true "
print(f'Input text:{input_text}')
max_sequence_length=model.input_shape[1]+1

predicted_word=predict_next_word(model,tokenizer,input_text,max_sequence_length)
print(f'Predicted word:{predicted_word}')

Input text:To be true 
Predicted word:a


In [23]:
model.save('next_word.h5')
#save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)